In [22]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
# from keras.preprocessing import image
from keras.applications.efficientnet import EfficientNetB0
from keras.applications.vgg16 import VGG16
import tensorflow_addons as tfa


In [25]:
# import os 
# os.chdir ('./Project/Code')

In [27]:
%pwd

'/Users/shreyesh/Desktop/Spring 2022/Neural Networks/Project/Code'

In [28]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Retrieve training data
# remove subset param
def getDataset(dataFolder, subset, imageSize = (224, 224), batchSize = 32):
    train_ds = keras.utils.image_dataset_from_directory(
      dataFolder,
      seed=123,
      image_size=imageSize,
      batch_size=batchSize)
    return train_ds

# Tune buffer size and efficiency 
# When do we call this?
def configurePerformance(train_ds, val_ds): 
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return train_ds, val_ds

# def buildModel(dropoutRate, numClasses, inpShape = (224, 224, 3)):
import pickle

# Create pkl file of the model after the training phase
def dumpModel(modelName, phase): 
    # Save the trained model as a pickle string.
    modelName = "model_" + modelName + "_ " + phase + ".pkl"
    pickle.dump(model, open(modelName, 'wb'))
    
def getDatasetsByCar(cars):
    train_ds = None 
    val_ds = None 
    for car in cars: 
        trainingFolder = 'data/'+ car +'/train/RGB/'
        testingFolder = 'data/'+ car + '/test_with_labels/RGB/'
        if not train_ds and not val_ds:
            train_ds = getDataset(trainingFolder, "training")
            val_ds =  getDataset(testingFolder, "validation")
        else: 
            new_train_ds =  getDataset(trainingFolder, "training")
            new_val_ds =  getDataset(testingFolder, "validation")
            train_ds.concatenate(new_train_ds)
            val_ds.concatenate(new_val_ds)
    return train_ds, val_ds


In [30]:
EPOCHS = 5
modelName = "vgg16"
# Initial layer input shape
inpShape =  (224, 224, 3)
cars = ['x5', 'model3']
train_ds, val_ds = getDatasetsByCar(cars)

# trainingFolder = 'data/x5/train/RGB/'
# testingFolder = 'data/x5/test_with_labels/RGB/'
# # don't need to pass subset string - datasets already split
# train_ds = getDataset(trainingFolder, "training")
# val_ds =  getDataset(testingFolder, "validation")

train_ds, val_ds = configurePerformance(train_ds, val_ds)

Found 6000 files belonging to 7 classes.
Found 1500 files belonging to 7 classes.
Found 6000 files belonging to 7 classes.
Found 1500 files belonging to 7 classes.


In [32]:
dropoutRate = 0.2
numClasses = 7 
inp = layers.Input(shape=inpShape)
baseModel = VGG16(weights="imagenet",
                   include_top = False) 
baseModel.trainable = False 
x = baseModel(inp, training=False)
x =  layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = layers.Dropout(dropoutRate, noise_shape=None, seed=None)(x)
out = layers.Dense(numClasses,activation="softmax", name = "pred")(x)
model = keras.Model(inp, out, name="FeatureExtraction-B0")
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
          # metrics=['accuracy']
          metrics=['accuracy',
              recall_m,
              precision_m,
              f1_m
              ]
          )

In [ ]:
# Feature extraction without the top layers 
hist_results = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)


Epoch 1/5


2022-04-15 12:21:39.052573: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-15 12:21:39.053135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - ETA: 0s - loss: 0.8400 - accuracy: 0.7997 - recall_m: 1.2673 - precision_m: 0.8042 - f1_m: 0.9778

2022-04-15 12:23:31.879010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - 139s 681ms/step - loss: 0.8400 - accuracy: 0.7997 - recall_m: 1.2673 - precision_m: 0.8042 - f1_m: 0.9778 - val_loss: 1.5861 - val_accuracy: 0.6553 - val_recall_m: 1.5036 - val_precision_m: 0.9691 - val_f1_m: 1.1726
Epoch 2/5
188/188 [==============================] - 138s 736ms/step - loss: 0.3315 - accuracy: 0.9020 - recall_m: 1.1355 - precision_m: 0.6950 - f1_m: 0.8572 - val_loss: 1.6377 - val_accuracy: 0.6507 - val_recall_m: 1.3456 - val_precision_m: 0.8379 - val_f1_m: 1.0276
Epoch 3/5
188/188 [==============================] - 141s 747ms/step - loss: 0.3002 - accuracy: 0.9162 - recall_m: 1.1120 - precision_m: 0.6753 - f1_m: 0.8354 - val_loss: 2.7379 - val_accuracy: 0.6387 - val_recall_m: 1.2760 - val_precision_m: 0.7755 - val_f1_m: 0.9603
Epoch 4/5
 40/188 [=====>........................] - ETA: 1:24 - loss: 0.2059 - accuracy: 0.9391 - recall_m: 1.0978 - precision_m: 0.6560 - f1_m: 0.8155

In [ ]:
dumpModel(modelName, "phase1")